
# Downloaded the data set from github :



In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip


In [ ]:
!unzip Flickr8k_Dataset.zip #Unziping the data set
!unzip Flickr8k_text.zip

In [ ]:
import numpy as np
import pandas as pd
from numpy import array
import matplotlib.pyplot as plt
%matplotlib inline

import string
import os
import glob
from PIL import Image
from time import time


from keras import Input, layers
from keras import optimizers
from keras.optimizers import Adam
from keras.preprocessing import sequence 
from keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import LSTM, Embedding , Dense , Activation , Flatten , Reshape , Dropout
from keras.layers import Bidirectional
from keras.layers import add
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from tensorflow.keras.utils import to_categorical
import pickle


Defining the token path. token path basicly contains the images and texts. and we can see in same Image there are 5 captions as below.


In [ ]:
token_path = "Flickr8k.token.txt"
train_images_path = "Flickr_8k.trainImages.txt"
test_images_path = "Flickr_8k.testImages.txt"
images_path = "Flicker8k_Dataset/"

doc = open(token_path,'r').read()
print(doc[:410])

In [ ]:
descriptions = dict()
for line in doc.split('\n'): # spliting the doc based on new line
  tokens = line.split()  # here spliting lines into tokens
  if len(line) > 2:
    image_id = tokens[0].split('.')[0] #spliting the only image id
    image_desc = ' '.join(tokens[1:])  # spliting the decsriptions 
    if image_id not in descriptions:
      descriptions[image_id] = list()
    descriptions[image_id].append(image_desc)
  


Here we have created a dictonary. my dict name is desriptions which has key and values. each key contains name of the image id and value conatins list of captions. 


In [ ]:
descriptions

Cleaning the text data:

In [ ]:
table = str.maketrans('','',string.punctuation)
for key,desc_list in descriptions.items():
  for i in range(len(desc_list)):
    desc = desc_list[i]
    desc = desc.split()
    desc = [word.lower() for word in desc]
    desc = [w.translate(table) for w in desc ]
    desc_list[i] = ' '.join(desc)


In [ ]:
# Checking the one image with its 5 captions
pic = '/content/Flicker8k_Dataset/101669240_b2d3e7f17b.jpg'
x = plt.imread(pic)
plt.imshow(x)
plt.show()
descriptions['101669240_b2d3e7f17b']

Counting the number of unique word or vlocabulary:

In [ ]:
vocabulary = set()
for key in descriptions.keys():
  [vocabulary.update(d.split()) for d in descriptions[key]]

print('Original Vocabulary Size : %d' % len(vocabulary))

In [ ]:
lines= list()
for key, desc_list in descriptions.items():
  for desc in desc_list:
    lines.append(key + ' '+ desc)
new_descriptins = '\n'.join(lines)
new_descriptins

Above we have made some changes. and created new descriptions. since  description contains key and value but that is not enough hence we have 
to make another variable which will contain seprate seprate description against same image id.

In [ ]:
doc = open(train_images_path,'r').read()
dataset = list()
for line in doc.split('\n'):
  if len(line) > 1:
    identifier = line.split('.')[0]
    dataset.append(identifier)


train = set(dataset)
train



Above we are creatimg the set which will be containg the all id's for traing. here we have removed extaintion "jpg".*   List item





Downlaoding the glove embeddings:

1.   Instead of creatimg embedding from scratc we are using glove embedding as
it has larger value and it understand the value in better way as compare to small embaddings.




In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip

In [ ]:
img = glob.glob(images_path+'*.jpg')
train_images = set(open(train_images_path,'r').read().strip().split('\n'))
train_img =[]
for i in img:
  if i[len(images_path):] in train_images:
    train_img.append(i)

print(train_img) # its path of train and test images
test_images = set(open(test_images_path,'r').read().strip().split('\n'))
test_img =[]
for i in img:
  if i[len(images_path):] in test_images:
    test_img.append(i)

Now we are adding the "start" and "end" token for each sentence. where "startseq" tell us where tio start and "endseq" tell us where to end the whole things. as soos as our model predicts end we know this time we have to stop and we can not pass the value to next LSTM. 

In [ ]:
train_descriptions = dict()
for line in new_descriptins.split('\n'):
  tokens = line.split()
  image_id,image_desc = tokens[0],tokens[1:]
  if image_id in train:
    if image_id not in train_descriptions:
      train_descriptions[image_id] = list()
    desc = 'startseq ' + ' '.join(image_desc)+' endseq'
    train_descriptions[image_id].append(desc)

In [ ]:
train_descriptions

In [ ]:
all_train_captions = []
for key,val in train_descriptions.items():
  for cap in val:
    all_train_captions.append(cap)

In [ ]:
all_train_captions

In [ ]:
#finding the vocabualty count, if any word occuring more then 10 times or more then 10 times, only that case only adding to vocabulary
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
  nsents+=1
  for w in sent.split(' '):
    word_counts[w] = word_counts.get(w,0) + 1
vocab = [w for w in word_counts if word_counts[w] >=word_count_threshold]

print('vocabulary = %d' % (len(vocab)))

In [ ]:
ixtoword = {}
wordtoix = {}
ix =1

for w in vocab:
  wordtoix[w] = ix
  ixtoword[ix] = w
  ix +=1
  

vocab_size = len(ixtoword) + 1

Here above instead using any tokenoizer we are creating index of each word.

In [ ]:
ixtoword

In [ ]:
# here finding the max length of captions
all_desc = list()
for key in train_descriptions.keys():
  [all_desc.append(d) for d in train_descriptions[key]]
lines = all_desc
max_length = max(len(d.split())for d in lines)

print("Description length : %d" % max_length)

Here we are loading the glove model and using glove file which has words or key- word desct.

In [ ]:
embeddings_index = {}
f = open(os.path.join('glove.6B.200d.txt'),encoding="utf-8")
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:],dtype='float32')
  embeddings_index[word] = coefs

In [ ]:
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size,embedding_dim)) # creatimg embedding vector 
for word, i in wordtoix.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
  

In [ ]:
embedding_matrix

**Defining the basic CNN mode - Inception**

In [ ]:
model = InceptionV3(weights='imagenet')

Since inception is classification model. this will be use as a classification but here we do not need clasification, we just need to extarct the features hence we are using the all the layers except the fully connected layers. and it will give all the features.


In [ ]:
model_new = Model(model.input,model.layers[-2].output)

**Preprocess for images**

In [ ]:
def preprocess(image_path):
  img = image.load_img(image_path,target_size=(299,299)) # loadinmg the image
  x = image.img_to_array(img) # conbverting into array
  x = np.expand_dims(x,axis=0) # expending the dimention because we are adding one more axis too it
  x = preprocess_input(x) # its keras function which we have imported above 
  return x

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:

# encoding the imgae where we will get encoded vectors
def encode(image):
  image = preprocess(image)
  fea_vec =  model_new.predict(image)
  fea_vec = np.reshape(fea_vec,fea_vec.shape[1])
  return fea_vec


# doing for train

encoding_train = {}
for img in train_img:
  encoding_train[img[len(images_path):]] = encode(img)
train_features = encoding_train

# doing for test
encoding_test = {}
for img in test_img:
  encoding_test[img[len(images_path):]] = encode(img)


In [ ]:
# saving the bottelneck tarin features
with open("/content/drive/MyDrive/encoded_train_images.pkl","wb") as encoded_pickle:
  pickle.dump(encoding_train,encoded_pickle)

In [ ]:
# saving the bottelneck test features
with open("/content/drive/MyDrive/encoded_test_images.pkl","wb") as encoded_pickle:
  pickle.dump(encoding_test,encoded_pickle)

In [ ]:
# defining the LSTM model 
inputs1 = Input(shape=(2048,)) # if we remove 2 last layers from inceptionv3 then the remaining layers would be 2048  
fe1 = Dropout(0.5)(inputs1) # droupout layers
fe2 = Dense(256, activation='relu')(fe1) # readucing 2048 to 256

# now passing through embedding layer
inputs2 = Input(shape=(max_length,)) 
se1 = Embedding(vocab_size,embedding_dim,mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2,se3]) # the value here which we got from CNN and LSTM , now we ware adding them
decoder2 = Dense(256,activation='relu')(decoder1) # and passing from another dense layer
outputs = Dense(vocab_size,activation='softmax')(decoder2) # passing through sofmax layer

model = Model(inputs=[inputs1,inputs2], outputs=outputs) 
model.summary()

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
def data_generator(descriptions,photos,wordtoix,max_length,num_photos_per_batch):
  X1,X2,y=list(),list(),list()
  n = 0
  #loop for ever over images
  while 1:
    for key,desc_list in descriptions.items():
      n +=1
      # retrive the photo feature
      photo = photos[key +'.jpg']
      for desc in desc_list:
        #encode the sequence
        seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
        # split one sequence into multipal X,y pairs
        for i in range(1,len(seq)):
          #split into input and output pair
          in_seq,out_seq=seq[:1],seq[i]
          #paid input sequence
          in_seq = pad_sequences([in_seq],maxlen=max_length)[0]
          #encode output sequence
          out_seq = to_categorical([out_seq],num_classes=vocab_size)[0]
          #store
          X1.append(photo)
          X2.append(in_seq)
          y.append(out_seq)



      if n==num_photos_per_batch:
        yield ([array(X1),array(X2)],array(y))
        X1,X2,y=list(),list(),list()
        n=0


In [ ]:
epochs = 30
batch_size = 3
steps = len(train_descriptions)//batch_size

generator = data_generator(train_descriptions,train_features,wordtoix,max_length,batch_size)
model.fit(generator,epochs=epochs,steps_per_epoch=steps,verbose=1)

In [ ]:
model.save('content/drive/MyDrive/model.h5')

In [ ]:
model.save_weights('content/drive/MyDrive/model_30.h5')

In [ ]:
def greedysearch(photo):
  in_text = 'startseq'
  for i in range(max_length):
    sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
    sequence = pad_sequences([sequence],maxlen=max_length)
    yhat = model.predict([photo,sequence],verbose=0)
    yhat = np.argmax(yhat)
    word = ixtoword[yhat]
    in_text += ' '+word
    if word == 'endseq':
      break

  final = in_text.split()
  final = final[1:-1]
  final = ' '.join(final)
  return final


In [ ]:
def beam_search_predictions(image,beam_index =3):
  start = [wordtoix["startseq"]]
  start_word = [[start,0.0]]
  while len(start_word[0][0]) < max_length:
    temp = []
    for s in start_word:
      per_caps = sequence.pad_sequences([s[0]], maxlen=max_length, padding='post')
      preds = model.predict([image,per_caps],verbose=0)
      word_preds = np.argsort(preds[0])[-beam_index:]
      #getting the top <beam_index>(n) predictions and creating a
      #new list so as to put them via the model again
      for w in word_preds:
        next_cap,prob=s[0][:],s[1]
        next_cap.append(w)
        temp.append([next_cap,prob])

    start_word = temp
    #sorting according to the probabilities
    start_word = sorted(star_word,reverse=Flase,key=lambda l : l[1])
    # getting the top words
    start_word = start_word[-beam_index:]
  start_word = start_word[-1][0]
  intermediate_caption = [ixtoword[i] for i in start_word]
  final_caption = []

  for i in intermediate_caption:
    if i != 'endseq':
      final_caption.append(i)
    else:
      break
        



  final_caption = ' '.join(final_caption[1:])
  return final_caption


In [ ]:
pic = '/content/Flicker8k_Dataset/101669240_b2d3e7f17b.jpg'
image = encoding_test[pic].reshape((1,2048))
x = plt.imread(images_path+pic)
plt.imshow(x)
plt.show()

print("Gready search:", greedysearch(image))
print("Beam Search, K =3:",beam_search_predictions(image,beam_index=3))
print("Beam Search, K =5:",beam_search_predictions(image,beam_index=5))
print("Beam Search, K =7:",beam_search_predictions(image,beam_index=7))
print("Beam Search, K =10:",beam_search_predictions(image,beam_index=10))